# Wrangling with the people dataframe

In [1]:
import pandas as pd
import numpy as np
import time

# Unpickle the people/votes dataframes

In [2]:
peopleFilepath = './Data/pickledPeople'
voteFilepath = './Data/pickledVotes_unpacked'
routeFilepath = './Data/pickledRoutes_unpacked'

people = pd.read_pickle(peopleFilepath)
votes = pd.read_pickle(voteFilepath)
routes = pd.read_pickle(routeFilepath)

In [3]:
routes.head(5)

,name,faInfo,commentCount,photoCount,latitude,longitude,diffRating,avgQualityRating,numQualityVotes,desc_aggregateData,...,bc1,bc2,bc3,bc4,bc5,bc6,bc7,bc8,bc9,bc10
https://www.mountainproject.com/route/105714687/bridalveil-falls-right-side,Bridalveil Falls Right Side,unknown,1,1,44.457798,-103.858901,5.9,3.2,5.0,"WI2+ Trad, Ice",...,https://www.mountainproject.com/area/105708963...,https://www.mountainproject.com/area/105714282...,https://www.mountainproject.com/area/105714675...,,,,,,,
https://www.mountainproject.com/route/105714691/honey-bear-main-flow,Honey Bear Main Flow,"Paul Piana, 1978/79",1,4,43.869700,-103.541302,None,2.2,6.0,WI3 Ice,...,https://www.mountainproject.com/area/105708963...,https://www.mountainproject.com/area/106058091...,https://www.mountainproject.com/area/105714306...,https://www.mountainproject.com/area/105714678...,,,,,,
https://www.mountainproject.com/route/105714695/terminator-pillar,Terminator Pillar,unknown,0,2,44.457798,-103.858901,5.9,3.7,6.0,"WI4 Trad, Ice",...,https://www.mountainproject.com/area/105708963...,https://www.mountainproject.com/area/105714282...,https://www.mountainproject.com/area/105714681...,,,,,,,
https://www.mountainproject.com/route/105714699/11th-hour-center,11th Hour Center,unknown,0,3,44.457798,-103.858901,5.10,2.3,9.0,"WI3 Trad, TR, Ice",...,https://www.mountainproject.com/area/105708963...,https://www.mountainproject.com/area/105714282...,https://www.mountainproject.com/area/105714684...,,,,,,,
https://www.mountainproject.com/route/105714703/gorillas-in-the-schist,Gorillas in the Schist,"John Walker, Dawn Glanc - Nov. 7, 2000",12,0,44.457798,-103.858901,5.9,1.5,2.0,"5.9 WI3 M5 X Trad, Mixed, Ice",...,https://www.mountainproject.com/area/105708963...,https://www.mountainproject.com/area/105714282...,https://www.mountainproject.com/area/105714675...,,,,,,,


In [4]:
# Sort people by the number of routes that they have given star ratings to
people = people.sort_values(by='numStarRatings', ascending=False)

# Finding People with Common Routes

In [ ]:
for i in people.iloc[0:50].index:
    print(i)

In [5]:
testGroup=[]
testGroup.append('https://www.mountainproject.com/user/10232/tony-b')
testGroup.append('https://www.mountainproject.com/user/10185/guy-h')
testGroup.append('https://www.mountainproject.com/user/106087077/mark-roth')
testGroup.append('https://www.mountainproject.com/user/105936209/khoa')
# testGroup.append('https://www.mountainproject.com/user/14537/slim')

# testGroup.append('https://www.mountainproject.com/user/13115/ken-duncan')
testGroup.append('https://www.mountainproject.com/user/11287/jay-eggleston')
# testGroup.append('https://www.mountainproject.com/user/13658/rob-bauer')       
testGroup.append('https://www.mountainproject.com/user/105890911/greg-davis')    
# testGroup.append('https://www.mountainproject.com/user/106192059/julius-beres')   # 10
# testGroup.append('https://www.mountainproject.com/user/106554608/andy-hansen')
# testGroup.append('https://www.mountainproject.com/user/105787348/dave-clawson')
# testGroup.append('https://www.mountainproject.com/user/10168/josh-janes')
# testGroup.append('https://www.mountainproject.com/user/14213/steven-lucarelli')

lst = set(people.loc[testGroup[0]]['starRatings'])
for i in range(len(testGroup)):
    lst = lst.intersection(set(people.loc[testGroup[i]]['starRatings']))
    
print("Common Routes:",len(lst))
# print()
# for i in lst:
#     print(i)

Common Routes: 188


In [6]:
# Add condition to continue past current user if percent not met by certain % of currentList scanned (should speed up significantly for large similarityDecimal values)
def find_users_with_common_routes(people, currentList, simThresh): # SimThresh is a decimal of what % of routes must match
    # Define variables to be used during calculation
    listLen = len(currentList)
    currentList = list(currentList)
    simUsers = []
    skipThresh1 = int(listLen*(1-simThresh))+1
    skipThresh2 = listLen*simThresh

    # Limit list of people to those with enough routes
    people = people.loc[people['numStarRatings']>=(listLen*simThresh)]
    peopleLen = len(people)
    
    # Find users with simThresh proportion of common routes
    for user in range(len(people)):
        print(user/peopleLen, end='\r')
        count = 0
        for route in range(len(currentList)):
            if currentList[route] in people.iloc[user]['starRatings']:
                count = count+1
            if route == skipThresh1:
                if count == 0:
                    break
        if count>=listLen*simThresh:
            simUsers.append(people.index[user])
            
    return simUsers

In [30]:
similarUsers = find_users_with_common_routes(people, lst, 0.6)

In [31]:
len(similarUsers)

14

In [32]:
set(similarUsers)-set(testGroup)

{'https://www.mountainproject.com/user/105795871/tombo',
 'https://www.mountainproject.com/user/105841324/dane-casterson',
 'https://www.mountainproject.com/user/106098789/john-a-tron',
 'https://www.mountainproject.com/user/106192059/julius-beres',
 'https://www.mountainproject.com/user/108386412/rick-vermeil',
 'https://www.mountainproject.com/user/13658/rob-bauer',
 'https://www.mountainproject.com/user/14015/nick-wilder',
 'https://www.mountainproject.com/user/14537/slim'}

In [77]:
testGroup=[]
testGroup.append('https://www.mountainproject.com/user/10232/tony-b')
testGroup.append('https://www.mountainproject.com/user/10185/guy-h')
testGroup.append('https://www.mountainproject.com/user/106087077/mark-roth')
testGroup.append('https://www.mountainproject.com/user/105936209/khoa')
testGroup.append('https://www.mountainproject.com/user/11287/jay-eggleston')
testGroup.append('https://www.mountainproject.com/user/105890911/greg-davis')
testGroup.append('https://www.mountainproject.com/user/105795871/tombo')
testGroup.append('https://www.mountainproject.com/user/105841324/dane-casterson')
testGroup.append('https://www.mountainproject.com/user/106098789/john-a-tron')
testGroup.append('https://www.mountainproject.com/user/106192059/julius-beres')
testGroup.append('https://www.mountainproject.com/user/108386412/rick-vermeil')
testGroup.append('https://www.mountainproject.com/user/13658/rob-bauer')
testGroup.append('https://www.mountainproject.com/user/14015/nick-wilder')
testGroup.append('https://www.mountainproject.com/user/14537/slim')

lst = set(people.loc[similarUsers[0]]['starRatings'])
for i in range(len(testGroup)):
    lst = lst.intersection(set(people.loc[similarUsers[i]]['starRatings']))
    
print("Common Routes:",len(lst))
print()
for i in lst:
    print(i)

Common Routes: 31

https://www.mountainproject.com/route/105748639/blind-faith
https://www.mountainproject.com/route/105749041/gonzo
https://www.mountainproject.com/route/105758257/the-clipboard
https://www.mountainproject.com/route/105748948/emerald-city
https://www.mountainproject.com/route/105749890/werk-supp
https://www.mountainproject.com/route/105750457/cosmosis
https://www.mountainproject.com/route/105748924/long-john-wall
https://www.mountainproject.com/route/105760950/kate-moss
https://www.mountainproject.com/route/105748047/east-face-standard
https://www.mountainproject.com/route/105750109/pool-of-blood
https://www.mountainproject.com/route/105756796/over-the-hill
https://www.mountainproject.com/route/105748951/the-unsaid
https://www.mountainproject.com/route/105750106/rewritten
https://www.mountainproject.com/route/105748774/ruper
https://www.mountainproject.com/route/105760935/dead-again
https://www.mountainproject.com/route/105749299/west-crack-aka-star-wars
https://www.mo

In [76]:
len(people.loc[people['numStarRatings']>500])

555

## Group 2 of people

In [80]:
testGroup=[]
testGroup.append('https://www.mountainproject.com/user/10788/c-miller')
testGroup.append('https://www.mountainproject.com/user/105908890/euan-cameron')

lst = set(people.loc[testGroup[0]]['starRatings'])
for i in range(len(testGroup)):
    lst = lst.intersection(set(people.loc[testGroup[i]]['starRatings']))
    
print("Common Routes:",len(lst))
# print()
# for i in lst:
#     print(i)

Common Routes: 650


In [81]:
similarUsers2 = find_users_with_common_routes(people, lst, 0.08)

KeyboardInterrupt: 

In [ ]:
lst = set(people.loc[similarUsers2[0]]['starRatings'])
for i in range(len(similarUsers2)):
    lst = lst.intersection(set(people.loc[similarUsers2[i]]['starRatings']))

In [ ]:
print("# Similar Users:", len(similarUsers2))
print("# Similar Routes:", len(lst))
print("List of Similar Users:")

newUsers2 = set(similarUsers2)-set(testGroup)
for i in newUsers2:
    print(i)

# Filtering by area

In [9]:
routes.head(5)

,name,faInfo,commentCount,photoCount,latitude,longitude,diffRating,avgQualityRating,numQualityVotes,desc_aggregateData,...,bc1,bc2,bc3,bc4,bc5,bc6,bc7,bc8,bc9,bc10
https://www.mountainproject.com/route/105714687/bridalveil-falls-right-side,Bridalveil Falls Right Side,unknown,1,1,44.457798,-103.858901,5.9,3.2,5.0,"WI2+ Trad, Ice",...,https://www.mountainproject.com/area/105708963...,https://www.mountainproject.com/area/105714282...,https://www.mountainproject.com/area/105714675...,,,,,,,
https://www.mountainproject.com/route/105714691/honey-bear-main-flow,Honey Bear Main Flow,"Paul Piana, 1978/79",1,4,43.869700,-103.541302,None,2.2,6.0,WI3 Ice,...,https://www.mountainproject.com/area/105708963...,https://www.mountainproject.com/area/106058091...,https://www.mountainproject.com/area/105714306...,https://www.mountainproject.com/area/105714678...,,,,,,
https://www.mountainproject.com/route/105714695/terminator-pillar,Terminator Pillar,unknown,0,2,44.457798,-103.858901,5.9,3.7,6.0,"WI4 Trad, Ice",...,https://www.mountainproject.com/area/105708963...,https://www.mountainproject.com/area/105714282...,https://www.mountainproject.com/area/105714681...,,,,,,,
https://www.mountainproject.com/route/105714699/11th-hour-center,11th Hour Center,unknown,0,3,44.457798,-103.858901,5.10,2.3,9.0,"WI3 Trad, TR, Ice",...,https://www.mountainproject.com/area/105708963...,https://www.mountainproject.com/area/105714282...,https://www.mountainproject.com/area/105714684...,,,,,,,
https://www.mountainproject.com/route/105714703/gorillas-in-the-schist,Gorillas in the Schist,"John Walker, Dawn Glanc - Nov. 7, 2000",12,0,44.457798,-103.858901,5.9,1.5,2.0,"5.9 WI3 M5 X Trad, Mixed, Ice",...,https://www.mountainproject.com/area/105708963...,https://www.mountainproject.com/area/105714282...,https://www.mountainproject.com/area/105714675...,,,,,,,


In [69]:
# Get top N most-rated routes at avalong
N=10

avalonRoutes = routes.loc[routes['bc4']=='https://www.mountainproject.com/area/105745528/avalon']
avalonRoutes = avalonRoutes.sort_values('numQualityVotes', ascending=False)
avalonRoutes = avalonRoutes[:N]

In [68]:
# Create a list of all users who have climbed the "N" most-rated route at Avalon
subPeople = set([])
            
for i in range(len(avalonRoutes)):
    tempSet=[]
    for j in range(len(people)):
        if avalonRoutes.index[i] in people.iloc[j]['starRatings']:
            tempSet.append(people.index[j])
            print(j/len(people), end='\r')        
    if i==0:
        subPeople=set(tempSet)
    else:
        subPeople = subPeople.intersection(set(tempSet))
            
print(len(subPeople),"have climbed the",N,"most-rated climbs at Avalon")

43 have climbed the 5 most-rated climbs at Avalon


# Function to find common people in an area

In [41]:
# Function to find all common people between N most-popular routes
def findCommonPeople(areaUrl, breadcrumbTier, nRoutes, routes, people):
    areaRoutes = routes.loc[routes[breadcrumbTier]==areaUrl]
    areaRoutes = areaRoutes.sort_values('numQualityVotes', ascending=False)
    areaRoutes = areaRoutes[:nRoutes]
    
    subPeople = set([])
    
    # Filter the people dataset to only include those with at least nRoutes starRatings
    people = people.loc[people['numStarRatings']>=nRoutes*5]
            
    for i in range(len(areaRoutes)):
        tempSet=[]
        print("Parsing peopleDF for route", i+1, "of", nRoutes)
        for j in range(len(people)):
            if areaRoutes.index[i] in people.iloc[j]['starRatings']:
                tempSet.append(people.index[j])
                print(j/len(people), end='\r')        
        if i==0:
            subPeople=set(tempSet)
        else:
            subPeople = subPeople.intersection(set(tempSet))
            
    print(len(subPeople),"have climbed the", nRoutes, "most-rated climbs at", areaUrl) 

    return subPeople

In [34]:
cali='https://www.mountainproject.com/area/105708959/california'
commonFolks = findCommonPeople(cali, 'bc1', 5, routes, people)

Parsing peopleDF for route 1 of 5
Parsing peopleDF for route 2 of 5
Parsing peopleDF for route 3 of 5
Parsing peopleDF for route 4 of 5
Parsing peopleDF for route 5 of 5
95 have climbed the 5 most-rated climbs at https://www.mountainproject.com/area/105708959/california


In [35]:
yos = 'https://www.mountainproject.com/area/105833381/yosemite-national-park'
commonFolks2 = findCommonPeople(yos, 'bc2', 10, routes, people)

Parsing peopleDF for route 1 of 10
Parsing peopleDF for route 2 of 10
Parsing peopleDF for route 3 of 10
Parsing peopleDF for route 4 of 10
Parsing peopleDF for route 5 of 10
Parsing peopleDF for route 6 of 10
Parsing peopleDF for route 7 of 10
Parsing peopleDF for route 8 of 10
Parsing peopleDF for route 9 of 10
Parsing peopleDF for route 10 of 10
33 have climbed the 10 most-rated climbs at https://www.mountainproject.com/area/105833381/yosemite-national-park


In [36]:
col = 'https://www.mountainproject.com/area/105708956/colorado'
commonFolks3 = findCommonPeople(col, 'bc1', 10, routes, people)

Parsing peopleDF for route 1 of 10
0.48417337597882436458Parsing peopleDF for route 2 of 10
Parsing peopleDF for route 3 of 10
Parsing peopleDF for route 4 of 10
Parsing peopleDF for route 5 of 10
Parsing peopleDF for route 6 of 10
Parsing peopleDF for route 7 of 10
Parsing peopleDF for route 8 of 10
Parsing peopleDF for route 9 of 10
Parsing peopleDF for route 10 of 10
26 have climbed the 10 most-rated climbs at https://www.mountainproject.com/area/105708956/colorado


In [43]:
eldo = 'https://www.mountainproject.com/area/105744246/eldorado-canyon-sp'
comFolks4 = findCommonPeople(eldo, 'bc3', 20, routes, people)

Parsing peopleDF for route 1 of 20
Parsing peopleDF for route 2 of 20
Parsing peopleDF for route 3 of 20
Parsing peopleDF for route 4 of 20
Parsing peopleDF for route 5 of 20
Parsing peopleDF for route 6 of 20
Parsing peopleDF for route 7 of 20
Parsing peopleDF for route 8 of 20
Parsing peopleDF for route 9 of 20
Parsing peopleDF for route 10 of 20
Parsing peopleDF for route 11 of 20
Parsing peopleDF for route 12 of 20
Parsing peopleDF for route 13 of 20
Parsing peopleDF for route 14 of 20
Parsing peopleDF for route 15 of 20
Parsing peopleDF for route 16 of 20
Parsing peopleDF for route 17 of 20
Parsing peopleDF for route 18 of 20
Parsing peopleDF for route 19 of 20
Parsing peopleDF for route 20 of 20
30 have climbed the 20 most-rated climbs at https://www.mountainproject.com/area/105744246/eldorado-canyon-sp


In [40]:
comFolks4 = findCommonPeople(eldo, 'bc3', 15, routes, people)

Parsing peopleDF for route 1 of 15
Parsing peopleDF for route 2 of 15
Parsing peopleDF for route 3 of 15
Parsing peopleDF for route 4 of 15
Parsing peopleDF for route 5 of 15
Parsing peopleDF for route 6 of 15
Parsing peopleDF for route 7 of 15
Parsing peopleDF for route 8 of 15
Parsing peopleDF for route 9 of 15
Parsing peopleDF for route 10 of 15
Parsing peopleDF for route 11 of 15
Parsing peopleDF for route 12 of 15
Parsing peopleDF for route 13 of 15
Parsing peopleDF for route 14 of 15
Parsing peopleDF for route 15 of 15
52 have climbed the 15 most-rated climbs at https://www.mountainproject.com/area/105744246/eldorado-canyon-sp


In [ ]:
calcOptimalRoutes = []
for i in range(5, 20):
    ppl = findCommonPeople(eldo, 'bc3', i, roites, people)
    calcOptimalRoutes.append(i, len(ppl))
